In [ ]:
import esf_utils
import os 
import pandas as pd 
import xml.etree.ElementTree as ET
import shutil
import configparser
import time
import datetime
import sys

print ( 'dependencies loaded.')

campaign_esf_paths_file = 'skaven_clan_skryre_me_one.txt'
campaign_files = esf_utils.parse_campaign_files_txt( campaign_esf_paths_file )


config = configparser.ConfigParser()
config.read('config.ini')

esf2xml_dir =  config['dependencies']['esf2xml']
save_folder = config['paths']['save_game_folder']
output_folder = config['paths']['output_folder']
extracted_subfolder = config['paths']['extracted_subfolder']
cwd = os.getcwd()



print('notebook initialized')


In [ ]:
# notebook for parsing a file 
econ_array = []
army_array = [] 

max = len(campaign_files)-1
i = 0 

def parse_regions(  extracted_output, new_array ) : 
    return None 

for s in campaign_files:

    save_file_clean = esf_utils.clean_filename( s )
    #'extract\\$filename_extract'

    out = output_folder
    templated = out.replace( '[$filename]', save_file_clean )
    
    full_path = os.path.join( save_folder , s )
    #output_dir =  f"{save_file_clean}_extract"
    file_modstamp = os.path.getmtime(full_path)
    ts = datetime.datetime.fromtimestamp( file_modstamp )
    unix_timestamp = int(time.mktime(ts.timetuple()))
    #print()
    
    try:
        dat1 = esf_utils.extract_save_file( 
        save_folder 
        , s
        ,templated
        ,config 
        )
    except:
        print("An exception occurred while extracting")
    

    extracted_output = os.path.join(templated, extracted_subfolder)
    
    try :
        session_id = esf_utils.get_session_guid( extracted_output )[1]
        turn_num = esf_utils.get_turn_number( extracted_output )
    
        # for faction economics at a high KPI level
        new_array = []
        esf_utils.parse_extracted_factions_folder( extracted_output, new_array ) 
        for r in new_array:
            r["session.id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            econ_array.append( r ) 


        # parse army information
        new_array = []
        #session_id, session_guid, turn_num = 
        esf_utils.parse_extracted_armies_folder( extracted_output, new_array ) 

        # write in all data universal to this save file
        for r in new_array:
            r["session.id"] = session_id
            r["turn_num"] = turn_num
            r["modifiedOn"] = unix_timestamp
            army_array.append( r )

        econ_df = pd.DataFrame( econ_array )
        army_df = pd.DataFrame( army_array )

        econ_df.to_csv('export_faction_economy.csv')
        army_df.to_csv('export_army_unit.csv')
    except:
        print("An exception occurred while extracting")
   
    print( f"@ { i }  / { max } folders loaded and exported")
    i += 1